In [14]:
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import statsmodels.tsa.stattools as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf ,plot_pacf
from pmdarima.arima import auto_arima
from pmdarima.preprocessing import FourierFeaturizer
import statsmodels.api as sma

%config IPCompleter.greedy=True
pd.set_option('display.max_rows', 50)

#Definimos la funcion para importar el JSON con los datos de las ventas
def import_data():
    with open("fishVigo.json") as file:
        dataset = pd.read_json(file)
        return dataset

#Asignamos el resultado de la funcion a una variable y lo convertimos a DataFrame
df = pd.DataFrame(import_data())
df.date = pd.to_datetime(df.date)

#Seteamos un indice basado en la fecha y la especie y eliminamos las filas duplicadas quedandonos con la primera de ellas
df = df.set_index(['date','name'])
df = df[~df.index.duplicated(keep = 'first')]

#Convertimos la columna de cantidades a numerico
df['quantity'] = df['quantity'].str.replace('Kg.','')
df['quantity'] = df['quantity'].str.replace('.','')
df['quantity'] = df['quantity'].str.replace(',','.')
df['quantity'] = pd.to_numeric(df['quantity'])
df = df['quantity']

#Mediante reshape creamos registros a 0. Ahora para todos los días tenemos datos de todas las especies
df = df.unstack().fillna(0).stack()
df.head

<bound method NDFrame.head of date        name               
2017-02-01  ABADEJO                 42.4
            ACEDIA-GOLLETAS          0.0
            ACEDIAS                  1.6
            AGUJAS                   0.0
            ALACHA                 310.0
                                   ...  
2019-09-09  VARIADO                 52.0
            VIEIRAS                  0.0
            VIEJAS                   0.0
            VOLADOR- POTA-POTON    125.0
            XOUBA                    0.0
Length: 172430, dtype: float64>

In [ ]:
class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        self.pValue = None
        self.isStationary = None
        
    def ADF_Stationarity_Test(self, timeseries, printResults = True):
        #Dickey-Fuller test:
        adfTest = sm.adfuller(timeseries, autolag='AIC')
        
        self.pValue = adfTest[1]
        
        if (self.pValue<self.SignificanceLevel):
            self.isStationary = True
        else:
            self.isStationary = False
        
        if printResults:
            dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
            #Add Critical Values
            for key,value in adfTest[4].items():
                dfResults['Critical Value (%s)'%key] = value
            print('Augmented Dickey-Fuller Test Results:')
            print(dfResults)

In [ ]:
#Sacamos el porcentaje de cuanto ha aportado al total de capturas cada especie
total = df.sum()
groupName = df.groupby('name').sum()
percentage = groupName/total*100
top = 5

#Seleccionamos aquellas especies cuyo aporte a las capturas supone mas del 5%
percentage = pd.DataFrame(percentage[percentage>top])
selectedSpecies = list(percentage.index.unique())
percentage.columns = ['quantity']
percentage = percentage.reset_index()

#Dibujamos
bar = go.Figure(go.Bar(x = percentage.name, y = percentage.quantity))
bar = bar.update_layout(title='Top '+str(top)+' Cuota',yaxis_title="% Porcentaje")
bar.show()

In [ ]:
#Extraemos del df aquellos datos pertenecientes a las especies seleccionadas
for x in selectedSpecies:
    dfTop = df.xs(x, level=1, drop_level=False)
    
    dfTop = pd.DataFrame(dfTop)
    
    dfTop.columns = ['quantity']
    dfTop = dfTop.reset_index()
    
    fig = go.Figure(data = go.Scatter(x=dfTop.date, y=dfTop.quantity),
                                        layout_title_text = x)
    fig.show()

In [ ]:
for x in selectedSpecies:
    df1 = df.xs(x, level=1, drop_level=False)

    sTest = StationarityTests()
    sTest.ADF_Stationarity_Test(df1, printResults = True)
    print("Is the time series " + x + " stationary? {0}".format(sTest.isStationary))

In [ ]:
dfPredict = df.xs('GALLO', level=1, drop_level=True)

dfPredict = dfPredict.asfreq('B',)

dfPredict[dfPredict.isnull()==True] = 0

model = sma.tsa.statespace.SARIMAX(dfPredict, order=(0, 1, 1), seasonal_order=(0, 1, 1, 12))
results = model.fit()
#plot = results.plot(500, 600)
results.plot_diagnostics(figsize=(18, 8))

#model.geterrors()

pred = results.get_prediction(start=pd.to_datetime('2018-01-03'), dynamic=False)
pred_ci = pred.conf_int()
pred_ci.plot()

In [ ]:
for x in selectedSpecies:
    y = "./plots/test" + x + ".png"
    
    dfDecomp = df.xs(x, level=1, drop_level=True)
    dfDecomp = dfDecomp.asfreq('B',)
    
    dfDecomp[dfDecomp.isnull()==True] = 0
    
    result = seasonal_decompose(dfDecomp, model='additive')
    fig = result.plot()
    fig.savefig(y,dpi=500)

In [ ]:
for x in selectedSpecies:
    filenameACF = "./plots/ACF-" + x + ".png"
    filenamePACF = "./plots/PACF-" + x + ".png"
    
    dfAutoCorrelation = df.xs(x, level=1, drop_level=True)
    dfAutoCorrelation = dfAutoCorrelation.asfreq('B',)
    
    dfAutoCorrelation[dfAutoCorrelation.isnull()==True] = 0
    
    figACF = plot_acf(dfAutoCorrelation,lags=150, zero=False)
    figPACF = plot_pacf(dfAutoCorrelation, lags=150, zero=False)
    figACF.savefig(filenameACF,dpi=500)
    figPACF.savefig(filenamePACF,dpi=500)
#Se puede observar en los graficos ACF y PACF que la seasonality es 5 (semanal en nuestro dataindex sin fines de semana), además podemos ver otras seasonalitys mas grandes, sin embargo varian dependiendo de la especie

In [ ]:
trans = FourierFeaturizer(65)
#exog = trans.fit_transform(dfARIMA)
#exog

In [ ]:
dfARIMA = df.xs('GALLO', level=1, drop_level=True)
dfARIMA = dfARIMA.asfreq('B',)

dfARIMA[dfARIMA.isnull()==True] = 0

stepwise_model = auto_arima(dfARIMA, start_p=1, start_q=1,
                           max_p=8, max_q=8, m=5,
                           seasonal=True,
                           d=None, D=None, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
print(stepwise_model.aic())
result = stepwise_model.fit(dfARIMA)

In [30]:
train, test = dfARIMA.loc[:'2019-01-06'], dfARIMA.loc['2019-01-07':]
result = stepwise_model.fit(train)
predction = result.predict_in_sample(test)

In [31]:
test.head()

date
2019-01-07    46164.52
2019-01-08    31111.52
2019-01-09    17091.90
2019-01-10    14033.50
2019-01-11    10529.30
Freq: B, dtype: float64